Vagrantプロビジョニングで設定できない項目の初期設定

# 前処理

## インポート

In [ ]:
from fabric import Connection, Config
import subprocess, re, time

## 関数 - ping応答待ち(Windows)

In [ ]:
def wait_ping(ip, timeout=60):
    for _ in range(timeout):
        if subprocess.run(["ping", "-n", "1", "-w", "1000", ip], stdout=subprocess.DEVNULL).returncode == 0:
            print('OK')
            return
        time.sleep(1)
    raise TimeoutError(f"{ip} unreachable")

## 接続情報
IPアドレスはDefault Switchで割り当てられたDHCPアドレスを取得

In [ ]:
remote_user = 'vagrant'
node_name   = 'ansible-node2'
key_file    = f'.vagrant/machines/{node_name}/hyperv/private_key'
new_address = '192.168.128.12'

In [ ]:
result = subprocess.run(["vagrant", "ssh-config", node_name], capture_output=True, text=True)
match = re.search(r'^\s*HostName\s+(.+)$', result.stdout, re.MULTILINE)
remote_host = match.group(1)
print(remote_host)

## fabric設定

In [ ]:
config = Config(overrides={
    'run': {'env': {'LANG': 'C'},'echo': True,'warn': True,'pty': False,'hide': False,'timeout': 20},
    'sudo': {'env': {'LANG': 'C'},'echo': True,'warn': True,'pty': True,'hide': False,'timeout': 20,'password': None},
    'timeouts': {'connect': 5,'command': 30,},
})

# 接続

## SSH接続

In [ ]:
c = Connection(
    port=22,
    host=remote_host,
    user=remote_user,
    connect_kwargs={
        'key_filename': key_file,
    },
    config=config,
)
c.run('hostname;date;whoami')

# 処理実行

## Hyper-V内部ネットワークのIPアドレス設定
プロビジョニング完了のタイミングでNICを割り当てしているためこのタイミングで設定

In [ ]:
c.run('ip a')

In [ ]:
#c.sudo(f'nmcli con mod eth1 ipv4.addresses {new_address}/24')
c.sudo(f'nmcli con add type ethernet ifname eth1 con-name eth1 ipv4.addresses {new_address}/24 ipv4.method manual')

In [ ]:
c.run('ip a')

In [ ]:
wait_ping(new_address)

In [ ]:
c.close()
c = Connection(
    port=22,
    host=new_address,
    user=remote_user,
    connect_kwargs={
        'key_filename': key_file,
    },
    config=config,
)
c.run('hostname;date;whoami')

## コントロールノードのSSH鍵を登録

In [ ]:
pubkey = open("id_rsa.pub").read().strip()

In [ ]:
c.run(f'echo "{pubkey}" >> ~/.ssh/authorized_keys')

# 切断

In [ ]:
c.close()